<div class="alert alert-block alert-info">
    <h1>Análisis de Series Temporales</h1>
    <h3>Clase 6</h3>
    <h3>Ejercicio 4</h3>
        <p>Docente: Rodrigo Del Rosso<p>
        <p>Asistentes: Sebastián Calcagno, Braian Drago <p>
</div>

RNN - Predicción en Mantenimiento - Compresor


**A continuación realizaremos:**  
Carga y visualización de datos originales  
Preparación adecuada de ventana de datos para alimentar un modelo de predicción    
Elaboración y entrenamiento de un modelo RNN   
Preparar ventanas de Calculo de predicciones finales, plot y métricas.

Importamos

In [ ]:
import pandas as pd
import tensorflow as tf   # Libreria clave para las Redes Neuronales Recurrentes
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
url='https://raw.githubusercontent.com/braiandrago/AST/main/DF_ALLIS_00.csv'
data = pd.read_csv(url, parse_dates=['DATE'])
data.set_index('DATE', inplace=True)
print(data.shape)
data.head()

(25489, 2)


,14TR_231_AGUA,14TR_232_PROD
DATE,,
1/1/2024 00:00:00,26.83,24.47
1/1/2024 00:10:00,26.76,24.40
1/1/2024 00:20:00,26.65,24.50
1/1/2024 00:30:00,26.50,24.47
1/1/2024 00:40:00,26.50,24.53


In [ ]:
data.tail()

,14TR_231_AGUA,14TR_232_PROD
DATE,,
25/6/2024 23:20:00,21.81,12.31
25/6/2024 23:30:00,21.84,12.35
25/6/2024 23:40:00,21.81,12.21
25/6/2024 23:50:00,21.88,11.99
26/6/2024 00:00:00,21.91,12.14


In [ ]:
# Graficar las series de tiempo con plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data['14TR_231_AGUA'], mode='lines', name='Temperatura Agua'))
fig.add_trace(go.Scatter(x=data.index, y=data['14TR_232_PROD'], mode='lines', name='Temperatura producto'))

fig.update_layout(
    title='Datos de Sensores a lo Largo del Tiempo',
    xaxis_title='Fecha',
    yaxis_title='Valores de Sensores',
    legend_title='Sensores'
)

fig.show()

In [ ]:
# Eliminar la columna '14TR_231_AGUA'
data.drop(columns=['14TR_231_AGUA'], inplace=True)
print(data.shape)
data.head()

(25489, 1)


,14TR_232_PROD
DATE,
1/1/2024 00:00:00,24.47
1/1/2024 00:10:00,24.40
1/1/2024 00:20:00,24.50
1/1/2024 00:30:00,24.47
1/1/2024 00:40:00,24.53


In [ ]:
# Calcula el punto de división basado en el 80% de los datos
split_time = int(len(data) * 0.8)

train_data = data[:split_time]
valid_data = data[split_time:]

print(f'Tamaño del conjunto de entrenamiento: {train_data.shape}')
print(f'Tamaño del conjunto de validación: {valid_data.shape}')

Tamaño del conjunto de entrenamiento: (20391, 1)
Tamaño del conjunto de validación: (5098, 1)


In [ ]:
# Crear la figura de Plotly para graficar entrenamiento y validacion
fig = go.Figure()


fig.add_trace(go.Scatter(x=train_data.index, y=train_data['14TR_232_PROD'], mode='lines', name='Temperatura producto'))


fig.add_trace(go.Scatter(x=valid_data.index, y=valid_data['14TR_232_PROD'], mode='lines', name='Temperatura producto'))

fig.update_layout(
    title='Serie temporal de entrenamiento y validación',
    xaxis_title='Dias',
    yaxis_title='Valores',
    legend_title='Series',

)

fig.show()

Prepara features y niveles

In [ ]:
# PArametros
window_size = 20
batch_size = 32


'''
Son dos parametros Claves:

window_size: Indica cuantos datos consecutivos se tomaran en cuenta para hacer una prediccion en una serie temporal para prever el siguiente valor.

batch_size=  Cantidad de ventanas procesadas al mismo tiempo durante el entrenamiento.

Aqui :
Procesamos 32 conjuntos de 20 dias de datos para entrenar el modelo.

'''

In [ ]:
'''
Aqui se toma la serie temporal y la se divide en ventanas de datos. Asi tenemos una secuencia de valores que se usan para predecir el siguiente valor en la serie.
'''


def windowed_dataset(series, window_size, batch_size):



    dataset = tf.data.Dataset.from_tensor_slices(series)


    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)

    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))

    dataset = dataset.map(lambda window: (window[:-1], window[-1]))

    dataset = dataset.batch(batch_size).prefetch(1)

    return dataset

In [ ]:
dataset = windowed_dataset(train_data, window_size, batch_size,)

In [ ]:
# Aqui se muestra la lista de arrays que representan las caracteristicas y las etiquetas de cada ventana en el conjunto de datos.
# Dependiendo del tamaño del batch_size y del window_size, cada línea impresa mostrará un lote de datos que el modelo utilizara para entrenamiento.
for window_dataset in dataset:
  print([item.numpy() for item in window_dataset])

Se han truncado las últimas 5000 líneas del flujo de salida.
        [11.71],
        [11.85],
        [11.75],
        [11.71],
        [11.25]],

       [[12.19],
        [12.05],
        [12.59],
        [12.29],
        [12.33],
        [12.51],
        [11.94],
        [12.33],
        [11.86],
        [11.83],
        [12.42],
        [11.93],
        [11.96],
        [11.59],
        [11.71],
        [11.85],
        [11.75],
        [11.71],
        [11.25],
        [11.13]],

       [[12.05],
        [12.59],
        [12.29],
        [12.33],
        [12.51],
        [11.94],
        [12.33],
        [11.86],
        [11.83],
        [12.42],
        [11.93],
        [11.96],
        [11.59],
        [11.71],
        [11.85],
        [11.75],
        [11.71],
        [11.25],
        [11.13],
        [10.85]],

       [[12.59],
        [12.29],
        [12.33],
        [12.51],
        [11.94],
        [12.33],
        [11.86],
        [11.83],
        [12.42],
        [11.93]

In [ ]:
# Imprimir propiedades de un solo lote
for windows in dataset.take(1):
  print(f'tipo de datos: {type(windows)}')
  print(f'numero de elementos de las tuplas: {len(windows)}')
  print(f'forma del primer elemento: {windows[0].shape}')
  print(f'forma del segundo elemento: {windows[1].shape}')

tipo de datos: <class 'tuple'>
numero de elementos de las tuplas: 2
forma del primer elemento: (32, 20, 1)
forma del segundo elemento: (32, 1)


A continuación, para la elaboración del modelo, tener en cuenta que utilizamos una *capa tipo LAMBDA*, la cual nos sirve para darle flexibilidad al modelo. Por ejemplo, en este caso *agregamos una dimensión* para adecuar el input al modelo de **Red Neuronal Recurrente Simple (SimpleRNN)**:

Las RNN necesitan recibir las entradas de forma que respeten la dimensión temporal de los datos, lo que significa que **necesitan un eje adicional que represente la secuencia temporal.**  

En el caso de la RNN, se agrega una dimensión adicional para cada paso de tiempo. El modelo puede procesar la secuencia valor por valor, teniendo en cuenta la interdependencia entre los pasos temporales.  

Cada **time step** (dentro del window size de 20) se procesa uno tras otro. Cada unidad de la RNN recibe la información del paso temporal anterior y la utiliza para actualizar su estado, permitiendo que el modelo capture dependencias temporales en la secuencia.

En una RNN, cada neurona en la red no solo recibe la entrada actual, sino también la salida del estado oculto anterior.

In [ ]:
# Aqui se entrena la red neuronal recurrente usando tensorFlow y keras
model = tf.keras.models.Sequential([
  tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                      input_shape=[window_size]),
  tf.keras.layers.SimpleRNN(40, return_sequences=True),
  tf.keras.layers.SimpleRNN(40),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 100.0) #para escalar la salida del modelo por un factor de 100 (solo info)
])

# Define un valor bajo q  controla cuanto cambian los pesos del modelo con respecto al error en cada iteracion.
'''
Uno bajo hace que el modelo aprenda de manera mas estable , pero + lento.
Uno alto permite que el modelo aprenda + rápido, pero con riesgo de inestabilidad.
'''
learning_rate = 1e-6


optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)


model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

# Entrena el modelo
history = model.fit(dataset,epochs=100)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/lambda_layer.py:65: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



637/637 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - loss: 9.0528 - mae: 9.5097
Epoch 2/100


/usr/lib/python3.10/contextlib.py:153: UserWarning:

Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.



637/637 ━━━━━━━━━━━━━━━━━━━━ 18s 14ms/step - loss: 3.9676 - mae: 4.3912
Epoch 3/100
637/637 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - loss: 2.1748 - mae: 2.5976
Epoch 4/100
637/637 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 1.4616 - mae: 1.8587
Epoch 5/100
637/637 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 2.1845 - mae: 2.6133
Epoch 6/100
637/637 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - loss: 1.8408 - mae: 2.2275
Epoch 7/100
637/637 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - loss: 1.9662 - mae: 2.3847
Epoch 8/100
637/637 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - loss: 1.4887 - mae: 1.8776
Epoch 9/100
637/637 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 2.0162 - mae: 2.4481
Epoch 10/100
637/637 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - loss: 1.7697 - mae: 2.1782
Epoch 11/100
637/637 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - loss: 1.9034 - mae: 2.3151
Epoch 12/100
637/637 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - loss: 2.0784 - mae: 2.4935
Epoch 13/100
637/637 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 2.0936 - mae: 2.4929


**Model Prediction**

In [ ]:
'''
Aqui vemos la shape de los primeros 20 datos de la serie  y finalmente utiliza el modelo para hacer una prediccion con los datos que han sido ajustados a la shape que el modelo espera.
'''
print(f'shape of series[0:20]: {data[0:20].shape}')


print(f'shape of series[0:20][np.newaxis]: {np.expand_dims(data[0:20], axis=0).shape}')


print(f'model prediction: {model.predict(np.expand_dims(data[0:20], axis=0))}')

shape of series[0:20]: (20, 1)
shape of series[0:20][np.newaxis]: (1, 20, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 454ms/step
model prediction: [[28.143793]]


In [ ]:
def model_forecast(model, series, window_size, batch_size):
    """
    Esta funcion es para realizar pronosticos de series temporales donde se quiere predecir un valor futuro basado en valores pasados dentro de un contexto de ventana.
    """


    dataset = tf.data.Dataset.from_tensor_slices(series)


    dataset = dataset.window(window_size, shift=1, drop_remainder=True)


    dataset = dataset.flat_map(lambda w: w.batch(window_size))


    dataset = dataset.batch(batch_size).prefetch(1)


    forecast = model.predict(dataset)

    return forecast

In [ ]:
'''
Aqui se esta reduciendo la serie temporal original para hacer predicciones y luego utiliza la funcion model_forecast para obtener esas predicciones
Facilitar el uso y la visualizacion de las predicciones al eliminar dimensiones innecesarias.
'''

# Reduce la serie original
forecast_series = data[split_time-window_size:-1]

# Hace las predicciones
forecast = model_forecast(model, forecast_series, window_size, batch_size)

# Eje unidimencional
results = forecast.squeeze()

160/160 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step


/usr/lib/python3.10/contextlib.py:153: UserWarning:

Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.



La diferencia clave entre forecast y results es la forma en que se representan los datos. forecast contiene las predicciones tal como las devuelve el modelo, mientras que results es una version mas simplificada y facil de usar para analisis o visualizacion

In [ ]:
forecast

array([[14.556505],
       [15.089689],
       [14.773085],
       ...,
       [12.816523],
       [12.413697],
       [12.302385]], dtype=float32)

In [ ]:
results

array([14.556505, 15.089689, 14.773085, ..., 12.816523, 12.413697,
       12.302385], dtype=float32)

In [ ]:
# Vista previa de formas después de usar los métodos de conversión y compresión
print(f'shape after converting to numpy array: {np.array(forecast).shape}')
print(f'shape after squeezing: {np.array(forecast).squeeze().shape}')

shape after converting to numpy array: (5098, 1)
shape after squeezing: (5098,)


In [ ]:
valid_convert=valid_data.squeeze()

# Se ejecuta esta linea para manejar mas facil los datos

In [ ]:
results.shape

(5098,)

In [ ]:
valid_convert.shape

(5098,)

**Calculámos algunas métricas**

In [ ]:
mse_metric = tf.keras.losses.MeanSquaredError()

# Calcular el MSE
mse = mse_metric(valid_convert, results).numpy()
print(mse)


18.472517


In [ ]:

mae_metric = tf.keras.metrics.MeanAbsoluteError()

mae_metric.update_state(valid_convert, results)

# Obtener el resultado de MAE
mae = mae_metric.result().numpy()
print(mae)


2.0059586


In [ ]:
import plotly.graph_objects as go


fig = go.Figure()

# Agregar datos de validación
fig.add_trace(go.Scatter(x=valid_data.index, y=valid_convert, mode='lines', name='Validación', line=dict(color='blue')))

# Agregar forecast
fig.add_trace(go.Scatter(x=valid_convert.index, y=results, mode='lines', name='Forecast', line=dict(color='cyan', dash='dash')))

# Configurar el diseño del gráfico
fig.update_layout(title='Validación vs Forecast', xaxis_title='Tiempo', yaxis_title='Valor')
fig.show()

**Realizamos algunas predicciones**

In [ ]:
def extend_predictions(model, series, window_size, batch_size, n_future):
    """
    Aqui el objetivo es extender las predicciones de un modelo de series temporales hacia el futuro
    """
    predictions = []
    current_window = series[-window_size:]

    for _ in range(n_future):

        input_window = np.expand_dims(current_window, axis=0)
        input_window = np.expand_dims(input_window, axis=-1)

        # Creamos un batch de tamaño 1
        input_batch = tf.data.Dataset.from_tensor_slices(input_window).batch(batch_size)

        # Predecimos el proximo valor
        next_value = model.predict(input_batch)[0, 0]
        predictions.append(next_value)

        # Actualiza la ventana actual con el nuevo valor.
        current_window = np.append(current_window[1:], next_value)

    return np.array(predictions)


In [ ]:
# Definir el número de pasos futuros
n_future = 5

data = data.values.flatten()

# predicciones futuras.
extended_predictions = extend_predictions(model, data, window_size, batch_size, n_future)

print(f'Extended Predictions: {extended_predictions}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Extended Predictions: [12.879377 14.062264 15.337422 16.599594 18.073816]


In [ ]:
extended_predictions

array([12.879377, 14.062264, 15.337422, 16.599594, 18.073816],
      dtype=float32)

In [ ]:
# GRaficamos
if not pd.api.types.is_datetime64_any_dtype(valid_data.index):
    valid_data.index = pd.to_datetime(valid_data.index, format='%d/%m/%Y %H:%M:%S', dayfirst=True)

start_forecast_date = valid_data.index[-1] + pd.Timedelta(minutes=10)

forecast_dates = pd.date_range(start=start_forecast_date, periods=n_future, freq='10min')  # Cambia '10T' a '10min'

fig = go.Figure()

fig.add_trace(go.Scatter(x=valid_data.index, y=valid_convert, mode='lines', name='Validación', line=dict(color='blue')))

fig.add_trace(go.Scatter(x=forecast_dates, y=extended_predictions, mode='lines', name='Predicción Extendida', line=dict(color='red', dash='dash')))

fig.update_layout(title='Validación vs Predicción Extendida', xaxis_title='Tiempo', yaxis_title='Valor')
fig.show()

